In [1]:
cd Downloads/jobathon

C:\Users\apurb\Downloads\jobathon


In [2]:
import numpy as np
import pandas as pd
from sklearn import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from xgboost import *
from sklearn.metrics import accuracy_score

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [4]:
df = pd.concat([train, test])
df.reset_index(drop=True,inplace=True)

In [5]:
df

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0.0
1,IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0.0
2,HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0.0
3,BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0.0
4,TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0.0
...,...,...,...,...,...,...,...,...,...,...,...
351032,DBENJOYI,Male,52,RG268,Salaried,X2,86,Yes,4242558,Yes,NaN
351033,CWQ72DWS,Male,55,RG277,Other,X2,86,Yes,1159153,No,NaN
351034,HDESC8GU,Male,35,RG254,Salaried,X4,15,No,1703727,No,NaN
351035,2PW4SFCA,Male,53,RG254,Other,X3,93,No,737178,Yes,NaN


In [6]:
df.isnull().sum()

ID                          0
Gender                      0
Age                         0
Region_Code                 0
Occupation                  0
Channel_Code                0
Vintage                     0
Credit_Product          41847
Avg_Account_Balance         0
Is_Active                   0
Is_Lead                105312
dtype: int64

In [7]:
df.drop('ID', axis=1, inplace=True)

In [8]:
df.Gender.unique()

array(['Female', 'Male'], dtype=object)

In [9]:
df = pd.get_dummies(df,columns=['Gender'])

In [10]:
df

,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,Gender_Female,Gender_Male
0,73,RG268,Other,X3,43,No,1045696,No,0.0,1,0
1,30,RG277,Salaried,X1,32,No,581988,No,0.0,1,0
2,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0.0,1,0
3,34,RG270,Salaried,X1,19,No,470454,No,0.0,0,1
4,30,RG282,Salaried,X1,33,No,886787,No,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
351032,52,RG268,Salaried,X2,86,Yes,4242558,Yes,NaN,0,1
351033,55,RG277,Other,X2,86,Yes,1159153,No,NaN,0,1
351034,35,RG254,Salaried,X4,15,No,1703727,No,NaN,0,1
351035,53,RG254,Other,X3,93,No,737178,Yes,NaN,0,1


In [11]:
df.Age.describe()

count    351037.000000
mean         43.860716
std          14.841932
min          23.000000
25%          30.000000
50%          43.000000
75%          54.000000
max          85.000000
Name: Age, dtype: float64

In [12]:
def age_cat(n):
    if 23<= n <=34:
        return 0
    elif 35<= n <=45:
        return 1
    elif 46<= n <=60:
        return 2
    else:
        return 3

In [13]:
df.Age = list(map(age_cat,df.Age))

In [14]:
df.Region_Code.unique()

array(['RG268', 'RG277', 'RG270', 'RG282', 'RG261', 'RG265', 'RG283',
       'RG254', 'RG269', 'RG257', 'RG279', 'RG280', 'RG252', 'RG284',
       'RG259', 'RG281', 'RG258', 'RG266', 'RG260', 'RG274', 'RG256',
       'RG275', 'RG273', 'RG267', 'RG272', 'RG251', 'RG262', 'RG264',
       'RG278', 'RG276', 'RG263', 'RG250', 'RG255', 'RG253', 'RG271'],
      dtype=object)

In [15]:
df = pd.get_dummies(df,columns=['Region_Code'])

In [16]:
df = pd.get_dummies(df,columns=['Occupation'])

In [17]:
df.Channel_Code.unique()

array(['X3', 'X1', 'X2', 'X4'], dtype=object)

In [18]:
df = pd.get_dummies(df,columns=['Channel_Code'])

In [19]:
df.Vintage.describe()

count    351037.000000
mean         46.924244
std          32.329303
min           7.000000
25%          20.000000
50%          32.000000
75%          73.000000
max         135.000000
Name: Vintage, dtype: float64

In [20]:
def vin_cat(n):
    if 7<= n <=39:
        return 0
    elif 40<= n <=72:
        return 1
    elif 73<= n <=105:
        return 2
    else:
        return 3

In [21]:
df.Vintage = list(map(age_cat,df.Vintage))

In [22]:
df.head(10)

,Age,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead,Gender_Female,Gender_Male,Region_Code_RG250,Region_Code_RG251,...,Region_Code_RG283,Region_Code_RG284,Occupation_Entrepreneur,Occupation_Other,Occupation_Salaried,Occupation_Self_Employed,Channel_Code_X1,Channel_Code_X2,Channel_Code_X3,Channel_Code_X4
0,3,1,No,1045696,No,0.0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,No,581988,No,0.0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
2,2,0,No,1484315,Yes,0.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,0,3,No,470454,No,0.0,0,1,0,0,...,0,0,0,0,1,0,1,0,0,0
4,0,0,No,886787,No,0.0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
5,2,0,No,544163,Yes,0.0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
6,3,3,NaN,1056750,Yes,1.0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
7,2,3,No,444724,Yes,0.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
8,1,1,No,1274284,No,0.0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,0
9,2,2,Yes,2014239,No,0.0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [23]:
df.Avg_Account_Balance.value_counts()

4022185    172
1611273     47
936814      45
4126262     44
1142030     43
          ... 
2199109      1
3769924      1
2250302      1
1738276      1
2320853      1
Name: Avg_Account_Balance, Length: 162137, dtype: int64

In [24]:
def aab_cat(n):
    if 20790<= n <=500000:
        return 0
    elif 600001<= n <=895000:
        return 1
    elif 895001<= n <= 1378000:
        return 2
    else:
        return 3

In [25]:
df.Avg_Account_Balance = list(map(aab_cat,df.Avg_Account_Balance))

In [26]:
df.Is_Active.value_counts()

No     214087
Yes    136950
Name: Is_Active, dtype: int64

In [27]:
df = pd.get_dummies(df,columns=['Is_Active'])

In [28]:
null_index = df[df['Credit_Product'].isnull()].index

In [29]:
null_index

Int64Index([     6,     15,     31,     36,     40,     43,     56,     60,
                61,     66,
            ...
            350950, 350962, 350964, 350971, 350983, 350985, 350991, 350997,
            351011, 351020],
           dtype='int64', length=41847)

In [30]:
cat_train_df = df.dropna()

In [31]:
cat_test_df = df.iloc[null_index]

In [32]:
cat_X_test = cat_test_df.drop(['Credit_Product','Is_Lead'],axis=1)

In [33]:
cat_train_df.Credit_Product.replace({'Yes':0, 'No':1}, inplace=True)

C:\Users\apurb\miniconda3\envs\snake\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [34]:
cat_xtrain, cat_xtest, cat_ytrain, cat_ytest = train_test_split(cat_train_df.drop(["Credit_Product","Is_Lead"], axis=1),cat_train_df.Credit_Product, test_size=0.25)

In [35]:
standard_scaler = preprocessing.StandardScaler()

In [36]:
cat_xtrain = standard_scaler.fit_transform(cat_xtrain)
cat_xtest = standard_scaler.fit_transform(cat_xtest)
cat_X_test = standard_scaler.fit_transform(cat_X_test)

In [37]:
credit_model = XGBClassifier(learning_rate =0.2,n_estimators=150,)

In [38]:
credit_model.fit(cat_xtrain, cat_ytrain)

C:\Users\apurb\miniconda3\envs\snake\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:02:40] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=150, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [39]:
predictions = credit_model.predict(cat_xtest)

In [40]:
accuracy_score(cat_ytest, predictions)

0.7272088724584104

In [41]:
df.Credit_Product.replace({'Yes':0, 'No':1}, inplace=True)

In [42]:
credit_missing = credit_model.predict(cat_X_test)

In [43]:
j=0
for i in null_index:
    df.iloc[i,2] = credit_missing[j]
    j+=1

# Model Testing

In [44]:
train_df = df.iloc[:245725,:]
test_df = df.iloc[245725:,:]

In [45]:
X = train_df.drop(["Is_Lead"], axis=1)
y = train_df["Is_Lead"]

In [46]:
model = XGBRegressor()

## Feature selection using sklearn's SelectKBest

In [47]:
bestfeatures = feature_selection.SelectKBest(score_func=feature_selection.chi2, k=30)

In [48]:
fit = bestfeatures.fit(X, y)
dfscore = pd.DataFrame(fit.scores_)
dfcolumn = pd.DataFrame(X.columns)
featureScore = pd.concat([dfcolumn,dfscore],axis=1)
featureScore.columns = ['Features','Score']

In [49]:
featureScore.sort_values('Score',ascending=False,inplace=True)
featureScore.reset_index(drop=True,inplace=True)

In [50]:
featureScore

,Features,Score
0,Age,15398.816790
1,Channel_Code_X1,12183.096547
2,Channel_Code_X3,6399.647674
3,Credit_Product,6377.384429
4,Vintage,4420.250026
5,Channel_Code_X2,3078.256689
6,Occupation_Entrepreneur,2643.079817
7,Occupation_Salaried,2392.408231
8,Is_Active_Yes,1104.406107
9,Occupation_Self_Employed,837.507017


In [52]:
top_14_features = featureScore.Features.to_list()[:14]

In [53]:
test_df_final = test_df[top_14_features]

In [54]:
X_train, X_test, y_train, y_test = train_test_split(train_df[top_14_features], y, test_size=0.3, random_state=42)

standard_scaler.fit(X_train)
X_train4 = standard_scaler.transform(X_train)
standard_scaler.fit(X_test)
X_test4 = standard_scaler.transform(X_test)
standard_scaler.fit(test_df_final)
test_df = standard_scaler.transform(test_df_final)

In [55]:
model.fit(X_train, y_train)
pred = model.predict(X_test)
metrics.roc_auc_score(y_test, pred)

0.7865556941555372

In [57]:
final_prediction = model.predict(test_df_final)

In [58]:
submissions = pd.DataFrame()

In [59]:
submissions['ID'] = test['ID']

In [60]:
submissions['Is_Lead'] = final_prediction

In [63]:
submissions.to_csv("Submission.csv")